# The TIF model iterator using pyspark

This notebook implements a tif iterator to use a sklearn model to predict every pixel in a raster .tif file.
This is done in a distributed way with pyspark because of the large number of pixels and time it will take without it.

It then outputs a segmentations shapefile which can be used in a GIS application to see the results.

Note MinMaxScalers have to be made in order to predict because of the differences between images.

Author: Michael de Winter

In [0]:
import rasterio
import pandas as pd
import itertools
import numpy as np
import glob
import joblib
from pyspark.sql.functions import udf
import pickle
from shapely.geometry import Polygon, Point
from timeit import default_timer as timer
import geopandas as gpd
import os
from pyspark.sql.types import DoubleType, StringType, ArrayType
import glob
import shutil

###### run the below command to import all the required functions

In [0]:
%run ./tif_model_iterator_functions

###Run settings

In [0]:
# Coepelduynen
path_to_output = "/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/"
path_to_scalers = "/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/scalers/"
parts = 2
model_path = "/dbfs/mnt/satellite-images-nso/models/randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.3.sav" 

In [0]:
check_done_files()

Out[15]: ['20190302',
 '20190302',
 '20190302',
 '20190302',
 '20190302',
 '20190302',
 '20190302',
 '20190302',
 '20190308',
 '20190308',
 '20190308',
 '20190308',
 '20190422',
 '20190422',
 '20190422',
 '20190422',
 '20190601',
 '20190601',
 '20190601',
 '20190601',
 '20191130',
 '20191130',
 '20191130',
 '20191130',
 '20191202',
 '20191202',
 '20191202',
 '20191202',
 '20200304',
 '20200304',
 '20200304',
 '20200304',
 '20200323',
 '20200323',
 '20200323',
 '20200323',
 '20200326',
 '20200326',
 '20200326',
 '20200326',
 '20200508',
 '20200508',
 '20200508',
 '20200508',
 '20200625',
 '20200625',
 '20200625',
 '20200625',
 '20200731',
 '20200731',
 '20200731',
 '20200731',
 '20200915',
 '20200915',
 '20200915',
 '20200915',
 '20201231',
 '20201231',
 '20201231',
 '20201231',
 '20210302',
 '20210302',
 '20210302',
 '20210302',
 '20210423',
 '20210423',
 '20210423',
 '20210423',
 '20210709',
 '20210709',
 '20210709',
 '20210709',
 '20210815',
 '20210815',
 '20210815',
 '20210815',
 '20210907',
 '20210907',
 '20210907',
 '20210907',
 '20220515',
 '20220515',
 '20220515',
 '20220515',
 '20220922',
 '20220922',
 '20220922',
 '20220922']

### Coepelduynen

In [0]:
for file  in glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/*.tif"):
  print(file.split("/")[-1].split("_")[0])

20190302
20190302
20190308
20190422
20190601
20191130
20191202
20200304
20200323
20200326
20200508
20200625
20200731
20200915
20201231
20210302
20210423
20210709
20210815
20210907
20211226
20220302
20220501
20220514
20220515
20220816
20220922

In [0]:
# Predict all 
for file  in glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/*.tif"):
  if file.split("/")[-1].split("_")[0] not in set(check_done_files()):
    try:
      print(file)
      run_tif_model_implementer(file,path_to_output,path_to_scalers,parts,model_path, aggregate_to_2m = False)
    except Exception as e:
      print(e)
  else:
      print(file.split("/")[-1].split("_")[0]+" already done")

20190302 already done
20190302 already done
20190308 already done
20190422 already done
20190601 already done
20191130 already done
20191202 already done
20200304 already done
20200323 already done
20200326 already done
20200508 already done
20200625 already done
20200731 already done
20200915 already done
20201231 already done
20210302 already done
20210423 already done
20210709 already done
20210815 already done
20210907 already done
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20211226_103526_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 8.513373872000102 second(s)
Filtering done in 38.381067487999644 second(s)
Filtered length of dataframe: 3535439
Normalization done in 38.55129070999965 second(s)
Finished making spark dataframe in 195.596199695 second(s)
Predicting finished in: 0.13339372999962507 second(s)
/databricks/spark/python/pyspark/sql/pandas/conversion.py:161: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
 An error occurred while calling o1335.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:428)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:107)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:103)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Job aborted due to stage failure: Task 20 in stage 0.0 failed 4 times, most recent failure: Lost task 20.3 in stage 0.0 (TID 55) (10.0.4.6 executor 0): org.apache.spark.api.python.PythonException: 'NameError: name 'loaded_model' is not defined', from <command-206828270476792>, line 135. Full traceback below:
Traceback (most recent call last):
 File "<command-206828270476792>", line 135, in predict_pandas_udf
NameError: name 'loaded_model' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:694)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:101)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:647)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknow

In [0]:
date_annotations = [20190601, 20200625, 20200731, 20200915, 20210709, 20210815,
       20210907, 20220515, 20220922]

In [0]:
check_done_files()

Out[18]: []

In [0]:
for date in date_annotations:
  print(date)
  
  file = glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/"+str(date)+"*.tif")[0]
  if file.split("/")[-1].split("_")[0] not in set(check_done_files()):
    print("File not done")
    run_tif_model_implementer(file,path_to_output,path_to_scalers,parts,model_path, aggregate_to_2m = False)
  else:
    print("File done")
    

20190601
File done
20200625
File done
20200731
File done
20200915
File done
20210709
File done
20210815
File done
20210907
File done
20220515
File not done
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 9.698784083999954 second(s)
Filtering done in 40.1953813990001 second(s)
Filtered length of dataframe: 3552354
Normalization done in 40.3807953160001 second(s)
Finished making spark dataframe in 183.56916892599997 second(s)
Predicting finished in: 0.5248957400001473 second(s)
Grouping labels finished in: 46.83219905200008 second(s)
Dissolving done in: 403.8340583239999 second(s)
/home/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Total permutations this step: 15411162
Done with extracting dataframe in 20.09839975399973 second(s)
Filtering done in 44.77717183300001 second(s)
Filtered length of dataframe: 3979837
Normalization done in 44.92628003799973 second(s)
Finished making spark dataframe in 198.99546538199957 second(s)
Predicting finished in: 0.04212436699981481 second(s)
Grouping labels finished in: 50.01575391999995 second(s)
Dissolving done in: 421.4746051070001 second(s)
/home/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
/home/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
/home/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Append
Done with merging files in: 903.0280544790003 second(s)
Writing to:/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.3.sav.shp
Done with whole run in: 2366.013574098 second(s)
20220922
File not done
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 9.077861441000096 second(s)
Filtering done in 40.10247311399962 second(s)
Filtered length of dataframe: 3553522
Normalization done in 40.237125320999894 second(s)
Finished making spark dataframe in 178.67396375199996 second(s)
Predicting finished in: 0.34041518899994117 second(s)
Grouping labels finished in: 45.67730325599996 second(s)
Dissolving done in: 785.9484591440005 second(s)
/home/20220922_110546_SV2-01_SV_RD_11bit_RGBI_50cm_Voorhout_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Total permutations this step: 15411162
Done with extracting dataframe in 21.516289228999995 second(s)
Filtering done in 44.41414065199933 second(s)
Filtered length of dataframe: 3979837
Normalization done in 44.55456310499994 second(s)
Finished making spark dataframe in 199.5694999819998 second(s)
Predicting finished in: 0.03834587599976658 second(s)
Grouping labels finished in: 53.04835336399992 second(s)
Dissolving done in: 474.0991576640008 second(s)
/home/20220922_110546_SV2-01_SV_RD_11bit_RGBI_50cm_Voorhout_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
/home/20220922_110546_SV2-01_SV_RD_11bit_RGBI_50cm_Voorhout_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
/home

In [0]:
for file  in glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/*"):
  print(file)

/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190308_111644_SV1-01_50cm_RD_11bit_RGBI_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190422_111335_SV1-01_50cm_RD_11bit_RGBI_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20191130_110721_SV1-01_50cm_RD_11bit_RGBI_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20191202_110525_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200304_114601_SV1-02_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200323_112115_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200326_114323_SV1-02_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200508_110812_SV1-01_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20201231_105943_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210302_111247_SV1-03_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210423_104948_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210709_103835_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210815_111051_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20211226_103526_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220302_112108_SV1-04_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220501_111418_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220514_114854_SV1-02_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220816_111150_SV2-01_SV_RD_11bit_RGBI_50cm_S

In [0]:
%sh

ls /dbfs/mnt/annotations/coepelduynen

Annotaties Coepelduynen 20190601.gpkg
Annotaties Coepelduynen 20200625.gpkg
Annotaties Coepelduynen 20200731.gpkg
Annotaties Coepelduynen 20200731.gpkg-shm
Annotaties Coepelduynen 20200731.gpkg-wal
Annotaties Coepelduynen 20200915.gpkg
Annotaties Coepelduynen 20200915.gpkg-shm
Annotaties Coepelduynen 20200915.gpkg-wal
Annotaties Coepelduynen 20210709.gpkg
Annotaties Coepelduynen 20210815.gpkg
Annotaties Coepelduynen 20210907.gpkg
Annotaties Coepelduynen 20220515.gpkg
Annotaties Coepelduynen 20220922.gpkg
annotaties_coepelduynen_to_pixel.csv

In [0]:
dates_annotations = [file.split(" ")[-1].split(".")[0] for file in glob.glob("/dbfs/mnt/annotations/coepelduynen/*.gpkg")]

In [0]:
dates_annotations

Out[6]: ['20190601',
 '20200625',
 '20200731',
 '20200915',
 '20210709',
 '20210815',
 '20210907',
 '20220515',
 '20220922']

In [0]:
dates_annotations.append("20200508")

In [0]:
dates_annotations

Out[11]: ['20190601',
 '20200625',
 '20200731',
 '20200915',
 '20210815',
 '20210907',
 '20220515',
 '20220922']

In [0]:
# For the annotations.
for date in dates_annotations:
  path_to_tif_file = glob.glob('/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/'+date+'*')[0]
  print(path_to_tif_file)
  run_tif_model_implementer(path_to_tif_file,path_to_output,path_to_scalers,parts,model_path)

/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 9.956320405000042 second(s)
Filtering done in 39.360326378000536 second(s)
Normalization done in 39.549338319000526 second(s)
Finished making spark dataframe in 190.2881780560001 second(s)
Predicting finished in: 0.36836192200007645 second(s)
 r g b ... label group_x group_y
0 0.265449 0.197120 0.163441 ... Bos 0.0 4802.0
1 0.263837 0.192169 0.160215 ... Bos 0.0 4802.0
2 0.265449 0.197120 0.163441 ... Bos 2.0 4802.0
3 0.263837 0.192169 0.160215 ... Bos 2.0 4802.0
4 0.268135 0.202520 0.169355 ... Bos 2.0 4798.0
... ... ... ... ... ... ... ...
3551971 0.258463 0.203420 0.162903 ... Gras 3198.0 2940.0
3551972 0.254702 0.198920 0.156989 ... Gras 3198.0 2940.0
3551973 0.252552 0.195770 0.152688 ... Gras 3198.0 2942.0
3551974 0.242343 0.184068 0.137634 ... Bos 3198.0 2942.0
3551975 0.230521 0.171017 0.121505 ... Bos 3198.0 2944.0

[3551976 rows x 13 columns]
Grouping to 2m finished in: 7.218818955000643 second(s)
/home/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Index(['geometry', 'label'], dtype='object')
Total permutations this step: 15411162
Done with extracting dataframe in 19.365496979000454 second(s)
Filtering done in 44.12743587600016 second(s)
Normalization done in 44.26805343200067 second(s)
Finished making spark dataframe in 204.66359082000054 second(s)
Predicting finished in: 0.04718929999944521 second(s)
 r g b ... label group_x group_y
0 0.322945 0.273177 0.265591 ... Gras 3200.0 1398.0
1 0.296077 0.242574 0.227419 ... Gras 3200.0 1400.0
2 0.279957 0.224572 0.204839 ... Gras 3200.0 1400.0
3 0.292853 0.238074 0.222581 ... Gras 3200.0 1402.0
4 0.311123 0.257876 0.248387 ... Gras 3200.0 1402.0
... ... ... ... ... ... ... ...
3981376 0.311123 0.252025 0.247312 ... Gras 6398.0 912.0
3981377 0.311123 0.252925 0.248387 ... Gras 6398.0 912.0
3981378 0.315422 0.258326 0.254839 ... Gras 6398.0 914.0
3981379 0.319183 0.262376 0.259677 ... Gras 6398.0 914.0
3981380 0.318109 0.260576 0.257527 ... Gras 6398.0 916.0

[3981381 rows x 13 columns]
Grouping to 2m finished in: 6.974870798999291 second(s)
/home/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Index(['geometry', 'label'], dtype='object')
/home/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
/home/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Append
Done with merging files in: 24.639111288000095 second(s)
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 8.51354127400009 second(s)
Filtering done in 39.0579912660005 second(s)
Normalization done in 39.17698162800025 second(s)
Finished making spark dataframe in 182.31897571900026 second(s)
Predicting finished in: 0.3539856070001

In [0]:
for date in dates_annotations:
  path_to_tif_file = glob.glob('/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/'+date+'*')[0]
  print(path_to_tif_file)

/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210815_111051_SV1-03_SV_RD_11bit_RGBI_50cm_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20220515_113347_SV1-02_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-741312215532814> in <module> 
 1 for date in dates_annotations : 
 ----> 2 path_to_tif_file = glob . glob ( '/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/' + date + '*' ) [ 0 ] 
 3 print ( path_to_tif_file ) 

 IndexError : list index out of range

In [0]:
run_tif_model_implementer("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif",path_to_output,path_to_scalers,parts,model_path, aggregate_to_2m = False)

/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 8.687650267999743 second(s)
Filtering done in 38.01366129300004 second(s)
Filtered length of dataframe: 3553522
Normalization done in 38.125182054000106 second(s)
Finished making spark dataframe in 183.10271757300052 second(s)
Predicting finished in: 0.32176552700002503 second(s)
 rd_x rd_y label
0 89908.75 472598.75 Bos
1 89909.25 472598.75 Bos
2 89906.75 472598.25 Bos
3 89907.25 472598.25 Gras
4 89907.75 472598.25 Bos
... ... ... ...
3553517 88977.75 470999.75 Struweel
3553518 88978.25 470999.75 Bos
3553519 88978.75 470999.75 Bos
3553520 88979.25 470999.75 Bos
3553521 88979.75 470999.75 Bos

[3553522 rows x 3 columns]
Grouping labels finished in: 52.716670160999456 second(s)
/home/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Index(['geometry', 'label'], dtype='object')
Total permutations this step: 15411162
Done with extracting dataframe in 19.84464542600108 second(s)
Filtering done in 41.987040443000296 second(s)
Filtered length of dataframe: 3979837
Normalization done in 42.10999645300035 second(s)
Finished making spark dataframe in 203.10164826500113 second(s)
Predicting finished in: 0.043726697000238346 second(s)
 rd_x rd_y label
0 88206.75 470998.75 Gras
1 88207.25 470998.75 Gras
2 88207.75 470998.75 Gras
3 88208.25 470998.75 Gras
4 88208.75 470998.75 Gras
... ... ... ...
3979832 87964.25 469400.75 Gras
3979833 87964.75 469400.75 Gras
3979834 87965.25 469400.75 Gras
3979835 87965.75 469400.75 Gras
3979836 87965.75 469400.25 Gras

[3979837 rows x 3 columns]
Grouping labels finished in: 56.88511645699873 second(s)
/home/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Index(['geometry', 'label'], dtype='object')
/home/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
/home/20210907_112017_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Append

In [0]:
import os

for file in glob.glob("/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/*v1.2*"):
  print(file)
  os.rename(file,file.split("_coepelduynen_contrast_annotations_")[0]+file.split("_coepelduynen_contrast_annotations_")[-1].split(".sav")[-1])

/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.cpg
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.dbf
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.shp
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.shx
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.cpg
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.dbf
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.shp
/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small_balanced_v1.2.sav.shx

In [0]:
for file in glob.glob("/dbfs/mnt/satellite-images-nso/model_out_coepelduynen/*v1.2*"):
  print(file)

In [0]:
# All files.
for path_to_tif_file in glob.glob("/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/*"):
  print(path_to_tif_file)
  run_tif_model_implementer(path_to_tif_file,path_to_output,path_to_scalers,parts,model_path, aggregate_to_2m= False)

/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 10.427086648000113 second(s)
Filtering done in 40.352837850998185 second(s)
Normalization done in 40.53532803900089 second(s)
Finished making spark dataframe in 189.97698182399836 second(s)
Predicting finished in: 6.262001445997157 second(s)
Grouping for output
/home/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Index(['geometry', 'label'], dtype='object')
Total permutations this step: 15411162
Done with extracting dataframe in 20.480283937999047 second(s)
Filtering done in 46.32159833699916 second(s)
Normalization done in 46.45904918800079 second(s)
Finished making spark dataframe in 208.53914567399988 second(s)
Predicting finished in: 6.2967404990013165 second(s)
Grouping for output
/home/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Index(['geometry', 'label'], dtype='object')
/home/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
/home/20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Append
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 warnings.warn(
Total permutations this step: 15415981
Done with extracting dataframe in 9.251330809001956 second(s)
Filtering done in 41.017747896999936 second(s)
Normalization done in 41.14325455500148 second(s)
Finished making spark dataframe in 187.6864122900006 second(s)
Predicting finished in: 6.733306923000782 second(s)
Grouping for output
/home/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
Index(['geometry', 'label'], dtype='object')
Total permutations this step: 15411162
Done with extracting dataframe in 20.885407263998786 second(s)
Filtering done in 44.833345695999014 second(s)
Normalization done in 44.97876207399895 second(s)
Finished making spark dataframe in 208.53120297499845 second(s)
Predicting finished in: 6.176030564998655 second(s)
Grouping for output
/home/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Index(['geometry', 'label'], dtype='object')
/home/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_1.shp
/home/20190302_105829_SV1-01_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_part_2.shp
Append
/dbfs/mnt/satellite-images-nso/SV_50cm/coepelduynen/20190308_111644_SV1-01_50cm_RD_11bit_RGBI_Oegstgeest_natura2000_coepelduynen_cropped_ndvi_height.tif
/databricks/python/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#secu

In [0]:
%sh

ls /dbfs/mnt/satellite-images-nso/model_out_coepelduynen

20190302_105726_SV1-01_50cm_RD_11bit_RGBI_Warmond_natura2000_coepelduynen_cropped_ndvi_height.tif_band2.save
20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.cpg
20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.dbf
20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shp
20190601_105844_SV1-04_50cm_RD_11bit_RGBI_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shx
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.cpg
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.dbf
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.shp
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height.shx
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.cpg
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.dbf
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shp
20200625_112015_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shx
20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.cpg
20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.dbf
20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shp
20200731_112003_SV1-03_SV_RD_11bit_RGBI_50cm_Rijnsburg_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shx
20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.cpg
20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.dbf
20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shp
20200915_112329_SV1-04_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height_randomforest_classifier_coepelduynen_contrast_annotations_grid_search_all_data_2019_2022_small.sav.shx
20210709_103835_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.cpg
20210709_103835_SV1-01_SV_RD_11bit_RGBI_50cm_KatwijkAanZee_natura2000_coepelduynen_cropped_ndvi_height.dbf
20210709_1038